# Cleans previously massaged data into separate csv files
This code uses the original public USGS and GreenStream data which was previously combined and pitch matched in terms of timings apache beam.  During that time the original data was also averaged from hourly reports into daily averages.

The data was averaged so that the model is generalized and not overly precise with outliers due to possible sensor inaccuracy. This is accpetable because we trying to virtualize water levels and due to terrain and flows there is no need to be overly precise.  Also this enables smaller dataset which is more suitable for faster model training.

In [32]:
# leave until I can determine if tensorboard runs in ibm cloud
# %load_ext tensorboard.notebook

In [58]:
import datetime
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import os
import os.path

In [59]:
CSV_FILE_NAME = "../data/usgs_gsvb_v2.csv"
os.environ['CSV_FILE_NAME'] = CSV_FILE_NAME

In [60]:
# The current directory will be where this src file is located.
# Which is in the src dir of the project
dirpath = os.getcwd()
print("current directory is : " + dirpath)

current directory is : /Users/davis/progs/ccivm/src


In [61]:
root_path = os.path.dirname(dirpath)
print("parent directory is : " + root_path)

parent directory is : /Users/davis/progs/ccivm


In [62]:
logs_path = os.path.join(root_path,"logs")
print("logs directory is : " + logs_path)

logs directory is : /Users/davis/progs/ccivm/logs


In [63]:
LOG_DIR_NAME = logs_path
os.environ['LOG_DIR_NAME'] = LOG_DIR_NAME

In [64]:
%%bash
echo ${LOG_DIR_NAME}

/Users/davis/progs/ccivm/logs


In [65]:
%%bash
head ${CSV_FILE_NAME}

usgs,site_no,date,tidal_high,tidal_low,precipitation_inches,lat,long
GSVB,33670001,2019-03-08,-6.7913385826771648,-6.7257217847769031,,36.735303,-76.011602
GSVB,33670001,2019-03-19,-6.8569553805774275,-6.758530183727034,,36.735303,-76.011602
GSVB,33670001,2019-02-21,1.9356955380577427,2.1325459317585302,,36.735303,-76.011602
GSVB,33670001,2019-03-11,-6.66010498687664,-6.4304461942257216,,36.735303,-76.011602
GSVB,33670001,2019-04-04,1.9028871391076114,2.0669291338582676,,36.735303,-76.011602
GSVB,33670001,2019-02-14,1.7716535433070866,1.9356955380577427,,36.735303,-76.011602
GSVB,33670001,2019-04-14,2.2309711286089238,2.39501312335958,,36.735303,-76.011602
GSVB,33670001,2019-02-24,2.65748031496063,2.9527559055118111,,36.735303,-76.011602
GSVB,33670001,2019-01-12,2.2965879265091864,2.39501312335958,,36.735303,-76.011602


In [66]:
%%bash
# this csv file has 9875 lines corresponding to records (plus one header row)
wc -l ${CSV_FILE_NAME}

    9876 ../data/usgs_gsvb_v2.csv


# Use generic python to read data into pandas dataframe

## In case of nans, 
In order to add default values for NA data, use the converters helper functions

## In case of missing data, 
In order to add default values for missing data use average value for column/feature average.  Averages were calculated seperately.

In [67]:
# Create a function to look for nans in the dataframe
# call via nans(dataset)
nans = lambda df: df[df.isnull().any(axis=1)]

In [68]:
# Anytime there is missing data for high tide column, use the default value of 1.2
def default_tidal_high(cell_value):
    # print(type(cell_value))
    # print("value is {}".format(cell_value))
    
    # Input cell values are strings
    # The blanks for missing values are litteral blanks
    try:
        ret_value = float(cell_value)
        return ret_value
    except:
        # Average of available data
        return float(1.2)
        
    return cell_value       

In [69]:
def default_tidal_low(cell_value):
    # print(type(cell_value))
    # print("value is {}".format(cell_value))
    
    # Input cell values are strings
    # The blanks for missing values are litteral blanks
    try:
        ret_value = float(cell_value)
        return ret_value
    except:
        # Average of available data
        return float(-0.89)
        
    return cell_value       

In [70]:
def default_precipitation(cell_value):
    # print(type(cell_value))
    # print("value is {}".format(cell_value))
    
    # Input cell values are strings
    # The blanks for missing values are litteral blanks
    try:
        ret_value = float(cell_value)
        return ret_value
    except:
        # the average rainfall divided by 365.
        return float(0.117)
        
    return cell_value       

In [71]:
raw_dataset = pd.read_csv(CSV_FILE_NAME, 
                          header=0,
                          parse_dates=[2],
                          converters={'precipitation_inches':default_precipitation, 
                                      'tidal_high':default_tidal_high,
                                      'tidal_low': default_tidal_low},
                          sep=",")

dataset = raw_dataset.copy()

## NaNs safety check

If still missing or NaNs exist, just drop them.  However, by this point none should exist.

In [72]:
# This should show an empty set
nans(dataset)

,usgs,site_no,date,tidal_high,tidal_low,precipitation_inches,lat,long


In [73]:
dataset.dropna(inplace=True)
nans(dataset)

,usgs,site_no,date,tidal_high,tidal_low,precipitation_inches,lat,long


In [74]:
dataset.shape
#dataset.head()

(9875, 8)

## column/feature cleanup

The columns for source of data, USGS vs GS is not needed.  We only care about water levels, not the source of the sensor reading.  Likewise, the site number column is unneeded.  The tidal_low is not of particlar interest since we are focusing on high water mark and not low mark.  The original greenstream data did not have a low tidal level anyway.  

The result is that a tidal high column and a regular water level from a Greenstream level mark have already been merged/pitch matched.

In [75]:
# drop any columns which we will not use
dataset.drop(['usgs','tidal_low','site_no'], axis=1, inplace=True)
dataset.head()

,date,tidal_high,precipitation_inches,lat,long
0,2019-03-08,-6.791339,0.117,36.735303,-76.011602
1,2019-03-19,-6.856955,0.117,36.735303,-76.011602
2,2019-02-21,1.935696,0.117,36.735303,-76.011602
3,2019-03-11,-6.660105,0.117,36.735303,-76.011602
4,2019-04-04,1.902887,0.117,36.735303,-76.011602


# Create new columns for year, month, day from the date column

In [76]:
def create_ymd_cols(df, column):
    df[column + "_year"] = df[column].apply(lambda x: x.year)
    df[column + "_month"] = df[column].apply(lambda x: x.month)
    df[column + "_day"] = df[column].apply(lambda x: x.day)
    
create_ymd_cols(dataset, 'date')

In [77]:
dataset.head()

,date,tidal_high,precipitation_inches,lat,long,date_year,date_month,date_day
0,2019-03-08,-6.791339,0.117,36.735303,-76.011602,2019,3,8
1,2019-03-19,-6.856955,0.117,36.735303,-76.011602,2019,3,19
2,2019-02-21,1.935696,0.117,36.735303,-76.011602,2019,2,21
3,2019-03-11,-6.660105,0.117,36.735303,-76.011602,2019,3,11
4,2019-04-04,1.902887,0.117,36.735303,-76.011602,2019,4,4


In [78]:
# remove the date column
dataset.pop('date')
dataset.head()

,tidal_high,precipitation_inches,lat,long,date_year,date_month,date_day
0,-6.791339,0.117,36.735303,-76.011602,2019,3,8
1,-6.856955,0.117,36.735303,-76.011602,2019,3,19
2,1.935696,0.117,36.735303,-76.011602,2019,2,21
3,-6.660105,0.117,36.735303,-76.011602,2019,3,11
4,1.902887,0.117,36.735303,-76.011602,2019,4,4


In [79]:
train_ds = dataset.sample(frac=0.7,random_state=3367)
leftover_ds = dataset.drop(train_ds.index)
test_ds = leftover_ds.sample(frac=0.5, random_state=3367)
valid_ds = leftover_ds.drop(test_ds.index)

In [84]:
# Create seperate series/dataframe? for labels
# Remove the label from the dataframes.

# TODO: Why drop this?
# TODO: also the normalized data should be 0-1. why do we have 
# 1.0+ values.
train_labels = train_ds.pop('tidal_high')
test_labels = test_ds.pop('tidal_high')
valid_labels = valid_ds.pop('tidal_high')

KeyError: 'tidal_high'

# Normalize the data

In [85]:
# use only the training data to get mean and std 
# which we can use to norm all datasets
train_stats_df = train_ds.copy()
train_stats = train_stats_df.describe().transpose()


In [86]:
def norm(a_df):
    # Do the norm operation
    normed_df = (a_df - train_stats['mean']) / train_stats['std']
    return normed_df

normed_train_ds = norm(train_ds)
normed_test_ds = norm(test_ds)
normed_valid_ds = norm(valid_ds)

In [87]:
#normed_train_ds
normed_valid_ds.head()

,precipitation_inches,lat,long,date_year,date_month,date_day
19,-0.031107,-0.869075,0.677893,1.540909,-1.467756,1.611953
21,-0.031107,-0.869075,0.677893,1.540909,-1.467756,-1.330050
24,-0.031107,-0.869075,0.677893,1.540909,-0.900735,-1.330050
25,-0.031107,-0.869075,0.677893,1.540909,-0.900735,0.480413
38,-0.031107,-0.869075,0.677893,1.540909,-1.467756,0.706721


# write the normalized datasets to disk